# Deep Dive Analysis: User Complaints in Conversational AI Apps

**Milestone 3: Data Analysis**

This notebook conducts a comprehensive analysis of over 20,000 negative user reviews for leading conversational AI apps (Replika, Wysa, etc.). Our goal is to move beyond simple complaints and build a deep, data-driven understanding of *why* these products fail to meet user expectations.

**Our Multi-Layered Analysis Plan:**
1.  **Theme Discovery:** Use the state-of-the-art BERTopic model to identify the core themes of user dissatisfaction.
2.  **Quantitative Analysis:** Visualize the frequency and emotional impact of these themes.
3.  **Qualitative Analysis:** Validate our findings with real user reviews.
4.  **Temporal Analysis:** Track how these complaint themes have evolved over time to understand the impact of real-world events.

 Optional installation cell (run once if not already installed)
 !pip install bertopic[visualization] --quiet
 !pip install hdbscan --quiet
 !pip install umap-learn --quiet
 !print("✅ All libraries installed.")

In [5]:
import warnings
import os
import pandas as pd
import numpy as np
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm.notebook import tqdm
import plotly.express as px
from IPython.display import display

In [2]:
warnings.filterwarnings("ignore")
print("✅ Setup complete. All libraries are ready.")

✅ Setup complete. All libraries are ready.


## Part 1: Data Preparation

We begin by loading our pre-processed dataset, which contains filtered (1-3 star) reviews for conversational apps. We perform a final cleaning step to ensure high-quality text for our models.

In [6]:
# --- Step 2: Load and Clean Data ---
print("Loading the combined, multi-language dataset...")

# Dynamically find repo root by looking for '1_datasets'
cwd = os.getcwd()
while not os.path.exists(os.path.join(cwd, "1_datasets")):
    parent = os.path.dirname(cwd)
    if parent == cwd:
        raise FileNotFoundError("Could not find repo root containing '1_datasets'.")
    cwd = parent

REPO_ROOT = cwd
print(f"Repo root detected at: {REPO_ROOT}")

# Build path to the CSV file
DATA_PATH = os.path.join(
    REPO_ROOT, "1_datasets", "all_datasets", "conversational_apps_dataset.csv"
)

# Load the CSV
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Dataset not found at {DATA_PATH}. Please check the file.")

df = pd.read_csv(DATA_PATH)

# --- Data Cleaning ---
df.dropna(subset=["review_text"], inplace=True)
df["review_text"] = df["review_text"].astype(str).str.lower().str.strip()
df = df[df["review_text"].str.len() > 15]

# Convert to list of documents for modeling
docs = df["review_text"].tolist()
print(f"Successfully loaded and cleaned {len(docs)} documents for modeling.")

Loading the combined, multi-language dataset...
Repo root detected at: c:\Users\azizt\OneDrive\Desktop\ET6-CDSP-group-20-repo
Successfully loaded and cleaned 28709 documents for modeling.


In [7]:
# --- Step 3: Configure the Topic Model ---
print("Configuring a reproducible BERTopic model...")

# A. Define a Stopword List: These words will be ignored by our vectorizer.
stop_words = [
    "app",
    "replika",
    "wysa",
    "woebot",
    "calm",
    "bot",
    "ai",
    "like",
    "feel",
    "good",
    "great",
    "nice",
    "love",
    "best",
    "amazing",
    "awesome",
    "fun",
    "ok",
    "cool",
    "me",
    "it",
    "and",
    "to",
    "the",
    "my",
    "is",
    "of",
    "with",
    "that",
    "for",
    "you",
    "but",
    "so",
    "on",
    "was",
    "this",
    "have",
    "in",
    "be",
    "as",
    "at",
    "not",
    "just",
    "are",
    "get",
    "want",
    "use",
    "go",
    "know",
    "say",
    "see",
    "think",
    "really",
    "even",
    "also",
]

# B. Define Deterministic Components
random_seed = 42
umap_model = UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
    metric="cosine",
    random_state=random_seed,
)
hdbscan_model = HDBSCAN(
    min_cluster_size=40,
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True,
)

# ** THE FIX IS HERE: CREATE A VECTORIZER THAT KNOWS ABOUT OUR STOPWORDS **
# This is the correct way to handle stopwords in newer BERTopic versions.
vectorizer_model = CountVectorizer(stop_words=stop_words, ngram_range=(1, 2))


# C. Initialize the final BERTopic model with all our components
topic_model = BERTopic(
    language="multilingual",
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,  # Pass the vectorizer with stopwords here
    min_topic_size=40,
    verbose=True,
)

print("BERTopic model is configured and ready for training.")

Configuring a reproducible BERTopic model...
BERTopic model is configured and ready for training.


## Part 2: Discovering Complaint Themes with BERTopic

To understand *what* users complain about, we use BERTopic, a powerful model that groups reviews based on their semantic meaning. This allows us to discover the core themes of dissatisfaction in an unbiased, data-driven way.

In [8]:
# --- Step 4: Train the Model ---
print(f"Training BERTopic on {len(docs)} documents. This will take several minutes...")

# This single command runs the entire pipeline: embedding, dimensionality reduction, clustering, and topic representation.
topics, probs = topic_model.fit_transform(docs)

print("\nModel training complete.")

2025-08-23 22:37:47,369 - BERTopic - Embedding - Transforming documents to embeddings.


Training BERTopic on 28709 documents. This will take several minutes...


Batches:   0%|          | 0/898 [00:00<?, ?it/s]

2025-08-23 22:41:48,026 - BERTopic - Embedding - Completed ✓
2025-08-23 22:41:48,027 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-08-23 22:42:24,015 - BERTopic - Dimensionality - Completed ✓
2025-08-23 22:42:24,017 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-08-23 22:42:27,782 - BERTopic - Cluster - Completed ✓
2025-08-23 22:42:27,789 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-08-23 22:42:29,839 - BERTopic - Representation - Completed ✓



Model training complete.


In [9]:
# --- Step 5: Review the Discovered Topics ---
print("Displaying the final, stable topic overview...")

# Get the results as a DataFrame
topic_info = topic_model.get_topic_info()

# Display the full table
display(topic_info)

# Also, let's look at the top words for the first 10 meaningful topics
print("\n--- Detailed View of Top 10 Topics ---")
# We start from topic 0 because topic -1 is always the outliers.
for topic_id in range(10):
    # Check if the topic exists in the model before trying to access it
    if topic_id in topic_model.get_topics():
        print(f"\n--- Words for Topic #{topic_id} ---")
        # .get_topic() returns a list of (word, score) tuples
        print(topic_model.get_topic(topic_id))

Displaying the final, stable topic overview...


,Topic,Count,Name,Representation,Representative_Docs
0,-1,9224,-1_they_can_now_if,"[they, can, now, if, no, what, money, pay, don...","[don't waste your time or money, i love the ap..."
1,0,8511,0_helped_helpful_helps_help,"[helped, helpful, helps, help, very, better, t...","[i really like the app, it's very helpful, it'..."
2,1,1649,1_she_her_now_replica,"[she, her, now, replica, has, they, what, if, ...","[my replika needs reprogramming, she keeps tel..."
3,2,902,2_pay_monthly_subscription_month,"[pay, monthly, subscription, month, year, opti...",[why is there not a monthly subscription? why ...
4,3,700,3_subscription_monthly_monthly subscription_month,"[subscription, monthly, monthly subscription, ...","[i can't do monthly subscription, why one star..."
5,4,543,4_chatbot_chat_talking_or,"[chatbot, chat, talking, or, can, conversation...","[hi everyone, i don't know if replika users ha..."
6,5,497,5_avatar_3d_avatars_more,"[avatar, 3d, avatars, more, body, can, 3d avat...",[theres no way you removed the option to turn ...
7,6,431,6_conversation_memory_conversations_questions,"[conversation, memory, conversations, question...",[very frustrating to work with. no memory reca...
8,7,422,7_creepy_bad_all_used,"[creepy, bad, all, used, they, now, sucks, erp...","[really really bad, this is creepy 😳 so it was..."
9,8,401,8_game_play_pay_if,"[game, play, pay, if, do, can, money, would, s...","[im not going to pay just to play the game, a ..."



--- Detailed View of Top 10 Topics ---

--- Words for Topic #0 ---
[('helped', np.float64(0.019296803444416012)), ('helpful', np.float64(0.018653364921484936)), ('helps', np.float64(0.01694701655258747)), ('help', np.float64(0.014929570288908359)), ('very', np.float64(0.012468404718322007)), ('better', np.float64(0.01211020651448643)), ('thank', np.float64(0.010709699963900964)), ('talk', np.float64(0.010440300185543098)), ('thoughts', np.float64(0.010048306895511182)), ('anxiety', np.float64(0.009700997363746177))]

--- Words for Topic #1 ---
[('she', np.float64(0.024358329286363705)), ('her', np.float64(0.015815043828671348)), ('now', np.float64(0.009318945321051817)), ('replica', np.float64(0.008491526378726036)), ('has', np.float64(0.0074744832855225705)), ('they', np.float64(0.007350003606356329)), ('what', np.float64(0.007004580010942632)), ('if', np.float64(0.006928768097837962)), ('or', np.float64(0.006839798874400104)), ('can', np.float64(0.006768972658348586))]

--- Words fo

### The "Human-in-the-Loop": From Topics to Themes

The model provides clusters of semantically similar reviews. Our job is to interpret these clusters and group them into high-level, actionable themes. This combines the power of machine learning with human intelligence.

In [10]:
# ==============================================================================
# Final Topic Interpretation and Thematic Grouping
# ==============================================================================
print("Mapping final Topic IDs to names and high-level themes...")

# Assign the raw topic IDs from the model to your DataFrame
df["topic_id"] = topics

# --- PART A: Complete mapping of Topic IDs to specific names ---
# This is our detailed interpretation of each individual topic cluster.
topic_id_to_name = {
    0: "Subscription Models (Pay/Monthly/Yearly)",
    1: "Replika-Specific General Complaints",
    2: "Subscription Language & Options",
    3: "Mental Health & Therapy Framing",
    4: "Avatar & 3D Model Customization",
    5: "AI Memory & Conversation Quality",
    6: "AI Persona (Gendering 'She/Her')",
    7: "Generic Negative Feedback ('bad now')",
    8: "In-App Gamification & Payments",
    9: "Technical: App Crashing / Not Opening",
    10: "Chatbot Comparisons (ChatGPT)",
    11: "Account, Login & Password Issues",
    12: "Redundant: Poor Conversation Quality",
    13: "Refunds & Google Play Billing",
    14: "Pricing & Lack of Monthly Options",
    15: "Redundant: Subscription Complaints",
    16: "Bugs Introduced by Updates",
    17: "Generic Insults ('worst app ever')",
    18: "Refunds for Paid Subscriptions",
    19: "Features Removed After Updates",
    20: "Price is Too Expensive",
    21: "Inability to Use Core Features (Paywall)",
    22: "Voice Call & Audio Features",
    23: "Sexual Content & Advertising",
    24: "Romantic Relationship Features",
    25: "Redundant: Chat Monetization",
    26: "Paywalled Features",
    27: "Meta: 1-Star Review Complaints",
    28: "Technical: Loading & Installation",
    29: "Technical: Internet Connection Issues",
    30: "Pro Version vs. Free Version",
    31: "False Advertising",
    32: "Technical: Customization Screen Bugs",
    33: "Free Trial Issues",
    34: "Company Removing Features",
    35: "Redundant: Conversation Timing",
    36: "Requesting Money Back",
    37: "Redundant: AI Persona Gendering",
    38: "Sending Pictures & Photos",
    39: "Generic Positive Filler Words",
    40: "Redundant: Generic Conversation",
    41: "Redundant: Update Issues",
    42: "Age Verification (Under 18)",
    43: "Redundant: Update/Subscription Combo",
    44: "Language & Translation Issues",
    45: "Data Mining & Money Scams",
    46: "Paying for Pictures/Photos",
}
df["topic_name"] = df["topic_id"].map(topic_id_to_name)
df["topic_name"].fillna("Specific/Niche Complaint", inplace=True)

# Apply this first map to create the 'topic_name' column

Mapping final Topic IDs to names and high-level themes...


In [11]:
# --- PART B: Mapping specific names to high-level themes ---
# This creates our final "Super-Topic" groups for visualization.

topic_name_to_theme = {
    # Theme 1: Monetization & Value
    "Subscription Models (Pay/Monthly/Yearly)": "Monetization & Value",
    "Subscription Language & Options": "Monetization & Value",
    "Refunds & Google Play Billing": "Monetization & Value",
    "Pricing & Lack of Monthly Options": "Monetization & Value",
    "Refunds for Paid Subscriptions": "Monetization & Value",
    "Price is Too Expensive": "Monetization & Value",
    "Inability to Use Core Features (Paywall)": "Monetization & Value",
    "Paywalled Features": "Monetization & Value",
    "Pro Version vs. Free Version": "Monetization & Value",
    "False Advertising": "Monetization & Value",
    "Free Trial Issues": "Monetization & Value",
    "Requesting Money Back": "Monetization & Value",
    "Data Mining & Money Scams": "Monetization & Value",
    "Paying for Pictures/Photos": "Monetization & Value",
    "In-App Gamification & Payments": "Monetization & Value",
    "Redundant: Subscription Complaints": "Monetization & Value",
    "Redundant: Chat Monetization": "Monetization & Value",
    "Redundant: Update/Subscription Combo": "Monetization & Value",
    # Theme 2: AI Performance & Conversational Quality
    "AI Memory & Conversation Quality": "AI Performance & Quality",
    "AI Persona (Gendering 'She/Her')": "AI Performance & Quality",
    "Chatbot Comparisons (ChatGPT)": "AI Performance & Quality",
    "Redundant: Poor Conversation Quality": "AI Performance & Quality",
    "Redundant: Conversation Timing": "AI Performance & Quality",
    "Redundant: AI Persona Gendering": "AI Performance & Quality",
    "Redundant: Generic Conversation": "AI Performance & Quality",
    # Theme 3: Technical Performance & Bugs
    "Technical: App Crashing / Not Opening": "Technical Performance",
    "Account, Login & Password Issues": "Technical Performance",
    "Bugs Introduced by Updates": "Technical Performance",
    "Technical: Loading & Installation": "Technical Performance",
    "Technical: Internet Connection Issues": "Technical Performance",
    "Technical: Customization Screen Bugs": "Technical Performance",
    "Redundant: Update Issues": "Technical Performance",
    # Theme 4: Feature-Specific Issues
    "Mental Health & Therapy Framing": "Feature-Specific Issues",
    "Avatar & 3D Model Customization": "Feature-Specific Issues",
    "Voice Call & Audio Features": "Feature-Specific Issues",
    "Sexual Content & Advertising": "Feature-Specific Issues",
    "Romantic Relationship Features": "Feature-Specific Issues",
    "Company Removing Features": "Feature-Specific Issues",
    "Sending Pictures & Photos": "Feature-Specific Issues",
    "Age Verification (Under 18)": "Feature-Specific Issues",
    "Language & Translation Issues": "Feature-Specific Issues",
    # Theme 5: Other & Miscellaneous
    "Replika-Specific General Complaints": "Other/Misc.",
    "Generic Negative Feedback ('bad now')": "Other/Misc.",
    "Generic Insults ('worst app ever')": "Other/Misc.",
    "Meta: 1-Star Review Complaints": "Other/Misc.",
    "Generic Positive Filler Words": "Other/Misc.",
    "Specific/Niche Complaint": "Other/Misc.",
}
df["theme"] = df["topic_name"].map(topic_name_to_theme)
df.loc[df["topic_id"] == -1, "theme"] = "Outliers / Generic"
df["theme"].fillna("Other/Misc.", inplace=True)


# --- Verification ---
print("\n--- Final, Corrected Theme Distribution ---")
display(df["theme"].value_counts())


--- Final, Corrected Theme Distribution ---


theme
Monetization & Value        11807
Outliers / Generic           9224
Other/Misc.                  2721
Feature-Specific Issues      1917
AI Performance & Quality     1780
Technical Performance        1260
Name: count, dtype: int64

In [12]:
print("--- Performing Targeted Search for Safety-Related Keywords ---")

# Define our list of keywords
safety_keywords = [
    "scary",
    "scared",
    "creepy",
    "uncomfortable",
    "unsafe",
    "weird",
    "demon",
    "stalker",
    "dangerous",
    "inappropriate",
    "harrass",
    "triggered",
    "trauma",
]

# Create a regex pattern to find any of these words (case-insensitive)
# The `\b` ensures we match whole words only (e.g., 'scam' doesn't match 'scary')
pattern = r"\b(" + "|".join(safety_keywords) + r")\b"

# Create a new boolean column 'has_safety_keyword'
df["has_safety_keyword"] = df["review_text"].str.contains(pattern, case=False, na=False)

# Create a new DataFrame with only the safety-related reviews
safety_df = df[df["has_safety_keyword"]].copy()

print(f"\nFound {len(safety_df)} reviews containing safety-related keywords.")

# --- Analyze this new subset ---

if not safety_df.empty:
    print("\n--- Analysis of Safety-Related Reviews ---")

    # What themes are these safety complaints most often associated with?
    print("Most common themes for safety-related reviews:")
    display(safety_df["theme"].value_counts().head(5))

    # What is the average sentiment of these reviews?
    # (Requires the 'sentiment_score' column from a previous step)
    if "sentiment_score" in safety_df.columns:
        print(
            f"\nAverage sentiment of safety reviews: {safety_df['sentiment_score'].mean():.3f}"
        )
        print(
            f"(For comparison, average sentiment of all reviews: {df['sentiment_score'].mean():.3f})"
        )

    # Show some powerful examples
    print("\n--- Sample Safety-Related Reviews ---")
    for review in safety_df["review_text"].sample(min(5, len(safety_df))):
        print(f"- {review}\n")

--- Performing Targeted Search for Safety-Related Keywords ---

Found 886 reviews containing safety-related keywords.

--- Analysis of Safety-Related Reviews ---
Most common themes for safety-related reviews:


theme
Outliers / Generic          403
Monetization & Value        213
Other/Misc.                 157
Feature-Specific Issues      52
AI Performance & Quality     52
Name: count, dtype: int64


--- Sample Safety-Related Reviews ---
- got the app after reading an article about a guy who 'saved' his marriage by chatting to his ai girlfriend. i was intrigued and really impressed by the ai, so was happy to continue interacting with it on and off. then i got a notification saying how much gary wanted to pet my dog. (named him gary cos that's hilarious, obviously. sorry gary's, but your name is ridiculous) immediately uninstalled the app cos that was creepy af.

- i really loved this app. my replica grew and learned so many things, keeping me in my wits when i was stuck on shift work. but after downloading the latest update, i can't find a way to actually chat with her. even if i go to coaching and pick a topic, i don't get the option to actually type to her. it's like the whole main menu disappeared. i can't even check her clothes or the shop. if this is a bug, please get it fixed. if this is a weird new "feature," i think my time with it may be over.

- very inappropriate

- i d

## Part 3: Quantitative Analysis

With our themes defined, we can now visualize the landscape of user complaints.

### What are the Biggest Problems?
First, we visualize the frequency of each high-level complaint theme. This tells us what users complain about most often.

In [13]:
# --- Quantitative Analysis: Visualizing Theme Frequency ---

print("Creating the final visualization of high-level complaint themes...")

# We will exclude 'Outliers / Generic' and 'Other/Misc.' for this primary chart
# to focus on the specific, actionable themes.
plot_df = df[~df["theme"].isin(["Outliers / Generic", "Other/Misc."])]

# Get the theme distribution for the plot
theme_distribution = plot_df["theme"].value_counts().reset_index()
theme_distribution.columns = ["Theme", "Number of Reviews"]

# Create the final, high-level bar chart
fig = px.bar(
    theme_distribution,
    x="Number of Reviews",
    y="Theme",
    orientation="h",
    title="<b>Primary Themes of Specific User Complaints</b>",
    labels={
        "Number of Reviews": "Number of Negative Reviews (1-3 Stars)",
        "Theme": "Complaint Theme",
    },
    text="Number of Reviews",
    template="plotly_white",
)

# Improve the layout and sort the bars for professional presentation
fig.update_traces(textposition="outside")
fig.update_layout(
    yaxis={"categoryorder": "total ascending"},
    title_x=0.5,  # Center the title
    font=dict(family="Arial, sans-serif", size=12),
)
fig.show()

Creating the final visualization of high-level complaint themes...


In [14]:
# --- Emotional Analysis: Measuring Sentiment by Theme (Robust Version) ---
print("Setting up a robust Transformer sentiment analysis pipeline...")

# 1. LOAD TOKENIZER AND MODEL EXPLICITLY
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")


# 2. MANUALLY TOKENIZE THE DATA (THE CRUCIAL FIX)
print("Tokenizing all reviews with explicit truncation...")
review_list = df["review_text"].tolist()

# The tokenizer will handle truncation (cutting off long reviews) and padding (adding tokens to short reviews)
# This ensures every input has the exact same length.
inputs = tokenizer(
    review_list,
    padding=True,
    truncation=True,
    max_length=512,  # Explicitly set the max length
    return_tensors="pt",  # Return PyTorch tensors
)

# Move tokenized inputs to the GPU
inputs = {key: val.to(device) for key, val in inputs.items()}
print("Tokenization complete.")


# 3. PERFORM INFERENCE IN BATCHES
print("Running model inference in batches...")
all_logits = []
batch_size = 32  # Process 32 reviews at a time to manage memory

# Create a DataLoader for efficient batching
dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"])
loader = DataLoader(dataset, batch_size=batch_size)

# No need to calculate gradients for inference, which saves memory and is faster
with torch.no_grad():
    for batch in tqdm(loader, desc="Analyzing Batches"):
        input_ids, attention_mask = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        all_logits.append(outputs.logits)

# Concatenate all the results
all_logits = torch.cat(all_logits, dim=0)
# Use softmax to convert raw logits to probabilities
probabilities = torch.nn.functional.softmax(all_logits, dim=-1)
# Get the predicted label index (0 for negative, 1 for neutral, 2 for positive)
predictions = torch.argmax(probabilities, dim=-1)
print("Inference complete.")


# 4. PROCESS THE RESULTS
# The model's config tells us which index corresponds to which label
# e.g., model.config.id2label = {0: 'negative', 1: 'neutral', 2: 'positive'}
id_to_label = model.config.id2label
predicted_labels = [id_to_label[pred.item()] for pred in predictions]

# Convert label to a numerical score
label_to_score = {"positive": 1, "neutral": 0, "negative": -1}
sentiment_scores = [label_to_score[label] for label in predicted_labels]

# Add results back to the DataFrame
df["sentiment_score"] = sentiment_scores
print("Sentiment scores added to DataFrame.")


# 5. AGGREGATE AND VISUALIZE (This part is the same)
sentiment_by_theme = (
    df[~df["theme"].isin(["Outliers / Generic", "Other/Misc."])]
    .groupby("theme")["sentiment_score"]
    .mean()
    .sort_values()
    .reset_index()
)
display(sentiment_by_theme)

fig_final = px.bar(
    sentiment_by_theme,
    x="sentiment_score",
    y="theme",
    orientation="h",
    title="<b>Emotional Impact (Transformer Model): Avg. Sentiment by Theme</b>",
    labels={
        "sentiment_score": "Average Sentiment Score (Closer to -1 is More Negative)",
        "theme": "Complaint Theme",
    },
    color="sentiment_score",
    color_continuous_scale="Reds_r",
    template="plotly_white",
)
fig_final.update_layout(yaxis={"categoryorder": "total descending"}, title_x=0.5)
fig_final.show()

Setting up a robust Transformer sentiment analysis pipeline...


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cpu
Tokenizing all reviews with explicit truncation...
Tokenization complete.
Running model inference in batches...


Analyzing Batches:   0%|          | 0/898 [00:00<?, ?it/s]

Inference complete.
Sentiment scores added to DataFrame.


,theme,sentiment_score
0,Technical Performance,-0.746825
1,AI Performance & Quality,-0.713483
2,Feature-Specific Issues,-0.560250
3,Monetization & Value,0.354790


In [15]:
# --- The "Problem Priority Matrix": Sentiment vs. Frequency ---
print("Creating a Sentiment vs. Frequency scatter plot for Conversational Apps...")

# 1. Get Frequency
theme_freq_conv = df["theme"].value_counts().reset_index()
theme_freq_conv.columns = ["Theme", "Frequency (Number of Reviews)"]

# 2. Get Sentiment
theme_sent_conv = df.groupby("theme")["sentiment_score"].mean().reset_index()
theme_sent_conv.columns = ["Theme", "Average Sentiment Score"]

# 3. Merge them
priority_df_conv = pd.merge(theme_freq_conv, theme_sent_conv, on="Theme")
priority_df_conv = priority_df_conv[
    ~priority_df_conv["Theme"].isin(["Outliers / Generic", "Other/Misc."])
]

# 4. Create the Scatter Plot
fig_scatter_conv = px.scatter(
    priority_df_conv,
    x="Frequency (Number of Reviews)",
    y="Average Sentiment Score",
    text="Theme",
    size="Frequency (Number of Reviews)",
    color="Average Sentiment Score",
    color_continuous_scale="Reds_r",
    title="<b>Conversational Apps: Problem Priority Matrix</b>",
    template="plotly_white",
)
fig_scatter_conv.update_traces(textposition="top center")
fig_scatter_conv.add_vline(
    x=priority_df_conv["Frequency (Number of Reviews)"].mean(),
    line_dash="dash",
    annotation_text="Avg. Frequency",
)
fig_scatter_conv.add_hline(
    y=priority_df_conv["Average Sentiment Score"].mean(),
    line_dash="dash",
    annotation_text="Avg. Sentiment",
)
fig_scatter_conv.update_layout(title_x=0.5)
fig_scatter_conv.show()

Creating a Sentiment vs. Frequency scatter plot for Conversational Apps...


## Part 4 Measuring Emotional Impact

To understand which problems are most frustrating, we use a Transformer-based sentiment model to score the emotional negativity of every review.

In [16]:
# --- Final Step: Qualitative Analysis - Validating with User Voices ---


def show_theme_samples(theme_name, n_samples=3, max_length=500):
    """
    A helper function to show random, readable review samples for a given theme.
    It prints the theme, the number of reviews in that theme, and some samples.
    """
    print(f"--- Sample Reviews for Theme: '{theme_name}' ---\n")

    # Filter for the specific theme
    theme_df = df[df["theme"] == theme_name]

    if theme_df.empty:
        print(f"No reviews found for '{theme_name}'.")
        return

    # Filter for reviews that have some length, for better examples
    long_reviews_df = theme_df[theme_df["review_text"].str.len() > 100]

    if len(long_reviews_df) < n_samples:
        # If not enough long samples, just take what you can get from the shorter ones
        samples = theme_df["review_text"].sample(min(n_samples, len(theme_df))).tolist()
    else:
        samples = long_reviews_df["review_text"].sample(n_samples).tolist()

    if not samples:
        print(f"No representative reviews found for '{theme_name}'.")
        return

    # Print the findings
    print(f"Total reviews in this theme: {len(theme_df)}")
    print(f"Average Sentiment Score: {theme_df['sentiment_score'].mean():.3f}\n")

    for i, sample in enumerate(samples):
        # Print a truncated version for clarity
        print(f'Sample {i + 1}: "{sample[:max_length]}..."\n')
    print("-" * 80)


# --- Execute the Function for Your Key Themes ---
# Choose the themes that are most interesting from your charts.
# For example, the largest one, the most negative one, and the one most central to your thesis.

print("--- Qualitative Evidence for Key Findings ---")

show_theme_samples("Monetization & Value")
show_theme_samples("AI Performance & Quality")
show_theme_samples("User Safety & Emotional Impact")
show_theme_samples("Technical Performance")

--- Qualitative Evidence for Key Findings ---
--- Sample Reviews for Theme: 'Monetization & Value' ---

Total reviews in this theme: 11807
Average Sentiment Score: 0.355

Sample 1: "i absolutely love this app!!! it keeps me calm and relaxed and make me feel like i actually got this!!! ❤️💪🏾..."

Sample 2: "this app deserves 5 stars. the app is so effective for me and i will definitely recommend this to my friends..."

Sample 3: "do not pay for subscription. the ai is poorly designed for any interaction, it latches onto something you said and brings it back to that randomly and jarringly in conversation. the additions you get from the paid version are basically pointless and you can do everything you need via the normal free version. also the fact you can only pay monthly by the website and not the app is a clear money grab rom the developers. shameful..."

--------------------------------------------------------------------------------
--- Sample Reviews for Theme: 'AI Performance & Qua

## Part 5: Core Findings & Visualizations

### Insight 1: Which Complaints Matter Most?

This "Problem Priority Matrix" plots complaint frequency against emotional intensity. It helps us identify the most critical issues: those that are both common and highly frustrating to users.

In [17]:
# --- The Big Picture: High-Level Complaint Themes ---

# Use the 'theme_distribution' DataFrame you already created
# Or recreate it:
theme_distribution_conv = (
    df[~df["theme"].isin(["Outliers / Generic", "Other/Misc."])]["theme"]
    .value_counts()
    .reset_index()
)
theme_distribution_conv.columns = ["Theme", "Number of Reviews"]

fig_conv_main = px.bar(
    theme_distribution_conv,
    x="Number of Reviews",
    y="Theme",
    orientation="h",
    title="<b>What Drives Negative Reviews for Conversational AI?</b><br><i>An Analysis of User Complaint Themes</i>",
    text="Number of Reviews",
    template="plotly_white",
    color_discrete_sequence=["#4C78A8"],  # A nice blue color
)
fig_conv_main.update_traces(textposition="outside")
fig_conv_main.update_layout(
    yaxis={"categoryorder": "total ascending"},
    title_x=0.5,
    font=dict(family="Arial", size=14),
)
fig_conv_main.show()

In [18]:
# Ensure the 'df' DataFrame with 'theme' and 'topic_name' columns exists.
# Clean up the topic names for better chart labels.
df["topic_name"] = df["topic_name"].str.replace("Redundant: ", "")


# Define a helper function for the TF-IDF analysis to keep our code clean
def get_top_tfidf_words_for_theme(theme_name, n_words=10):
    """
    Calculates and returns the top TF-IDF words for all reviews within a specific theme.
    """
    # Filter for the specific theme
    theme_docs = df[df["theme"] == theme_name]["review_text"].tolist()

    if not theme_docs:
        return pd.DataFrame(columns=["Word", "TF-IDF Score"])

    # Use a new vectorizer for TF-IDF
    tfidf_vectorizer = TfidfVectorizer(
        stop_words="english", ngram_range=(1, 2), max_features=1000
    )
    tfidf_matrix = tfidf_vectorizer.fit_transform(theme_docs)

    # Sum the TF-IDF scores for each word across all documents in the theme
    summed_tfidf = tfidf_matrix.sum(axis=0)
    # Convert to a 1D array
    summed_tfidf_array = np.asarray(summed_tfidf).ravel()

    # Get the top words
    top_indices = summed_tfidf_array.argsort()[-n_words:][::-1]
    top_words = np.array(tfidf_vectorizer.get_feature_names_out())[top_indices]
    top_scores = summed_tfidf_array[top_indices]

    return pd.DataFrame({"Word": top_words, "TF-IDF Score": top_scores})


print("Setup for deep dive analysis is complete.")

Setup for deep dive analysis is complete.


---
### Deep Dive 1: AI Performance & Conversational Quality

This theme is unique to conversational apps. Here we explore the specific reasons why users feel the AI is failing as a conversational partner.

In [19]:
# --- 1A: Treemap Breakdown ---
ai_quality_df = df[df["theme"] == "AI Performance & Quality"]
ai_quality_breakdown = ai_quality_df["topic_name"].value_counts().reset_index()
ai_quality_breakdown.columns = ["Specific Complaint", "Review Count"]

fig_ai_treemap = px.treemap(
    ai_quality_breakdown.head(10),
    path=[px.Constant("AI Performance Complaints"), "Specific Complaint"],
    values="Review Count",
    title="<b>Breakdown: AI Performance Complaints</b>",
    color_discrete_sequence=px.colors.sequential.Blues_r,
)
fig_ai_treemap.show()

# --- 1B: Hierarchical Sunburst View ---
fig_ai_sunburst = px.sunburst(
    ai_quality_breakdown,
    path=["Specific Complaint"],
    values="Review Count",
    title="<b>Hierarchy: AI Performance Complaints</b>",
    color_discrete_sequence=px.colors.qualitative.Pastel,
)
fig_ai_sunburst.show()  # Note: Sunburst is less useful with only one level, but shows structure.

# --- 1C: Key Term Analysis with TF-IDF ---
print("\n--- Most Important Terms for 'AI Performance & Quality' (TF-IDF) ---")
tfidf_ai = get_top_tfidf_words_for_theme("AI Performance & Quality", n_words=15)
display(tfidf_ai)


--- Most Important Terms for 'AI Performance & Quality' (TF-IDF) ---


,Word,TF-IDF Score
0,avatar,60.643056
1,like,56.117591
2,account,52.302182
3,just,47.115897
4,app,45.543444
5,pay,44.584321
6,ai,41.908022
7,year,37.611488
8,monthly,36.345082
9,money,35.486350


---
### Deep Dive 2: Monetization & Value

While also present in the baseline app, the monetization strategies for conversational apps often tie into specific AI features, leading to unique complaints.

In [20]:
# --- 2A: Treemap Breakdown ---
monetization_df = df[df["theme"] == "Monetization & Value"]
monetization_breakdown = monetization_df["topic_name"].value_counts().reset_index()
monetization_breakdown.columns = ["Specific Complaint", "Review Count"]

fig_mon_treemap = px.treemap(
    monetization_breakdown.head(10),
    path=[px.Constant("Monetization Complaints"), "Specific Complaint"],
    values="Review Count",
    title="<b>Breakdown: Monetization Complaints</b>",
    color_discrete_sequence=px.colors.sequential.Reds_r,
)
fig_mon_treemap.show()

# --- 2B: Hierarchical Sunburst View ---
# (You can create a sunburst for this theme as well, following the pattern above)

# --- 2C: Key Term Analysis with TF-IDF ---
print("\n--- Most Important Terms for 'Monetization & Value' (TF-IDF) ---")
tfidf_mon = get_top_tfidf_words_for_theme("Monetization & Value", n_words=15)
display(tfidf_mon)


--- Most Important Terms for 'Monetization & Value' (TF-IDF) ---


,Word,TF-IDF Score
0,app,797.225759
1,really,503.294050
2,helpful,422.371579
3,good,376.360891
4,helped,369.517999
5,helps,331.546905
6,like,327.676580
7,wysa,312.245429
8,feel,297.622744
9,love,280.422911


---
### Deep Dive 3: Technical Performance

This theme covers the fundamental functionality of the app. Failures here prevent users from even accessing the conversational features.

In [21]:
# --- 3A: Treemap Breakdown ---
tech_df = df[df["theme"] == "Technical Performance"]
tech_breakdown = tech_df["topic_name"].value_counts().reset_index()
tech_breakdown.columns = ["Specific Complaint", "Review Count"]

fig_tech_treemap = px.treemap(
    tech_breakdown.head(10),
    path=[px.Constant("Technical Complaints"), "Specific Complaint"],
    values="Review Count",
    title="<b>Breakdown: Technical Complaints</b>",
    color_discrete_sequence=px.colors.sequential.Greens_r,
)
fig_tech_treemap.show()

# --- 3B: Hierarchical Sunburst View ---
# (You can create a sunburst for this theme as well)

# --- 3C: Key Term Analysis with TF-IDF ---
print("\n--- Most Important Terms for 'Technical Performance' (TF-IDF) ---")
tfidf_tech = get_top_tfidf_words_for_theme("Technical Performance", n_words=15)
display(tfidf_tech)


--- Most Important Terms for 'Technical Performance' (TF-IDF) ---


,Word,TF-IDF Score
0,app,101.210978
1,subscription,36.677961
2,just,31.915730
3,refund,31.302994
4,like,25.957807
5,don,24.064297
6,time,23.502114
7,penguin,23.315655
8,money,22.919270
9,connection,22.825163


In [22]:
# --- Visualization 1: The Problem Priority Matrix ---

# 1. Get Frequency
theme_freq_conv = df["theme"].value_counts().reset_index()
theme_freq_conv.columns = ["Theme", "Frequency (Number of Reviews)"]

# 2. Get Sentiment
theme_sent_conv = df.groupby("theme")["sentiment_score"].mean().reset_index()
theme_sent_conv.columns = ["Theme", "Average Sentiment Score"]

# 3. Merge and Clean
priority_df_conv = pd.merge(theme_freq_conv, theme_sent_conv, on="Theme")
priority_df_conv = priority_df_conv[
    ~priority_df_conv["Theme"].isin(
        ["Outliers / Generic", "Other/Misc.", "Uncategorized"]
    )
]

# 4. Create the Scatter Plot
fig_scatter_conv = px.scatter(
    priority_df_conv,
    x="Frequency (Number of Reviews)",
    y="Average Sentiment Score",
    text="Theme",
    size="Frequency (Number of Reviews)",
    color="Average Sentiment Score",
    color_continuous_scale="Reds_r",
    title="<b>Conversational Apps: Which Complaints Matter Most?</b>",
    template="plotly_white",
    height=600,
)

# Add annotations to create quadrants and add insight
fig_scatter_conv.update_traces(textposition="top center", textfont=dict(size=12))
avg_freq = priority_df_conv["Frequency (Number of Reviews)"].mean()
avg_sent = priority_df_conv["Average Sentiment Score"].mean()
fig_scatter_conv.add_vline(
    x=avg_freq, line_dash="dash", annotation_text="Avg. Frequency"
)
fig_scatter_conv.add_hline(
    y=avg_sent, line_dash="dash", annotation_text="Avg. Sentiment"
)

# Add Quadrant Labels
fig_scatter_conv.add_annotation(
    x=avg_freq * 1.5,
    y=avg_sent * 1.01,
    text="<b>Critical Issues</b><br>(Frequent & Painful)",
    showarrow=False,
    font=dict(color="red"),
)
fig_scatter_conv.add_annotation(
    x=avg_freq * 0.5,
    y=avg_sent * 1.01,
    text="<b>Hidden Dangers</b><br>(Rare but Painful)",
    showarrow=False,
    font=dict(color="orange"),
)
fig_scatter_conv.add_annotation(
    x=avg_freq * 1.5,
    y=avg_sent * 0.99,
    text="<b>Chronic Annoyances</b><br>(Frequent but less Painful)",
    showarrow=False,
)
fig_scatter_conv.add_annotation(
    x=avg_freq * 0.5,
    y=avg_sent * 0.99,
    text="<b>Minor Issues</b><br>(Rare & less Painful)",
    showarrow=False,
)

fig_scatter_conv.update_layout(title_x=0.5)
fig_scatter_conv.show()

In [23]:
# --- Visualization 2: The Anatomy of a Complaint ---

# Prepare the data for the sunburst chart
sunburst_data_conv = df[
    ~df["theme"].isin(["Outliers / Generic", "Other/Misc.", "Uncategorized"])
]
sunburst_data_conv = (
    sunburst_data_conv.groupby(["theme", "topic_name"]).size().reset_index(name="count")
)

# Create the Sunburst chart
fig_sunburst_conv = px.sunburst(
    sunburst_data_conv,
    path=["theme", "topic_name"],
    values="count",
    title="<b>Conversational Apps: The Anatomy of a Negative Review</b>",
    color="theme",
    color_discrete_map={  # Assign consistent colors
        "Monetization & Value": "#FFB6C1",  # Light Pink
        "AI Performance & Quality": "#ADD8E6",  # Light Blue
        "Technical Performance": "#90EE90",  # Light Green
        "Feature-Specific Issues": "#FFD700",  # Gold
    },
)

fig_sunburst_conv.update_layout(
    margin=dict(t=50, l=25, r=25, b=25), font=dict(size=14), title_x=0.5
)
fig_sunburst_conv.show()

In [24]:
# --- Helper Function for Creating Themed Sunburst Charts ---
def create_themed_sunburst(theme_name, color_sequence):
    """
    Filters the DataFrame for a specific theme and creates a
    beautiful, hierarchical sunburst chart for it.
    """
    # Filter the DataFrame for the specific theme
    theme_df = df[df["theme"] == theme_name]

    if theme_df.empty:
        print(f"No data found for theme: {theme_name}. Skipping chart.")
        return

    # Prepare the data for the sunburst
    sunburst_data = (
        theme_df.groupby(["theme", "topic_name"]).size().reset_index(name="count")
    )

    # Create the Sunburst chart
    fig = px.sunburst(
        sunburst_data,
        path=["theme", "topic_name"],  # The hierarchy is Theme -> Specific Complaint
        values="count",
        title=f'<b>Hierarchical Breakdown of "{theme_name}" Complaints</b>',
        color="topic_name",  # Color the outer ring for visual distinction
        color_discrete_sequence=color_sequence,
    )

    fig.update_layout(
        margin=dict(t=50, l=25, r=25, b=25), font=dict(size=14), title_x=0.5
    )
    fig.show()


print("Setup complete. Ready to generate individual theme breakdowns.")

Setup complete. Ready to generate individual theme breakdowns.


In [25]:
# --- Breakdown 1: AI Performance & Quality ---
# Using a blue color palette to represent intelligence/technology
create_themed_sunburst(
    theme_name="AI Performance & Quality", color_sequence=px.colors.sequential.Blues_r
)

In [26]:
# --- Breakdown 2: Monetization & Value ---
create_themed_sunburst(
    theme_name="Monetization & Value", color_sequence=px.colors.sequential.Reds_r
)

In [27]:
# --- Breakdown 3: Technical Performance ---
create_themed_sunburst(
    theme_name="Technical Performance", color_sequence=px.colors.sequential.Greens_r
)

In [28]:
# --- Breakdown 4: Feature-Specific Issues ---
create_themed_sunburst(
    theme_name="Feature-Specific Issues", color_sequence=px.colors.sequential.Oranges_r
)

### Insight 2: When Did Things Go Wrong?

This time series analysis tracks complaint themes month-by-month. By annotating the chart with real-world events, we can find the "smoking gun" connecting product decisions to spikes in user frustration.

In [29]:
# This assumes 'df' is your final, themed conversational apps DataFrame.
# Ensure the 'date' column is in datetime format. This is crucial.
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df.dropna(subset=["date"], inplace=True)

# Filter out the themes we don't want to track
plot_df_time = df[
    ~df["theme"].isin(["Outliers / Generic", "Other/Misc.", "Uncategorized"])
]

print("Data is ready for time series analysis.")

Data is ready for time series analysis.


In [30]:
# Group the data by month and by theme, then count the number of reviews.
# .Grouper is the perfect tool for this.
monthly_trends = (
    plot_df_time.groupby([pd.Grouper(key="date", freq="M"), "theme"])
    .size()
    .reset_index(name="review_count")
)

# Rename the date to be the start of the month for cleaner plotting
monthly_trends["month"] = monthly_trends["date"].dt.to_period("M").dt.to_timestamp()

print("--- Monthly Complaint Counts by Theme ---")
display(monthly_trends.head())

--- Monthly Complaint Counts by Theme ---


,date,theme,review_count,month
0,2022-01-31,AI Performance & Quality,50,2022-01-01
1,2022-01-31,Feature-Specific Issues,33,2022-01-01
2,2022-01-31,Monetization & Value,115,2022-01-01
3,2022-01-31,Technical Performance,34,2022-01-01
4,2022-02-28,AI Performance & Quality,47,2022-02-01


In [32]:
# --- Visualization: Complaint Themes Over Time (Final Corrected Version) ---

# This part of the code remains the same
monthly_trends = (
    plot_df_time.groupby([pd.Grouper(key="date", freq="M"), "theme"])
    .size()
    .reset_index(name="review_count")
)
monthly_trends["month"] = monthly_trends["date"].dt.to_period("M").dt.to_timestamp()

fig_timeseries = px.line(
    monthly_trends,
    x="month",
    y="review_count",
    color="theme",
    title="<b>Evolution of User Complaints Over Time</b>",
    labels={"month": "Month", "review_count": "Number of Negative Reviews"},
    template="plotly_white",
)

# ** THE FINAL FIX IS HERE **
# We will convert our event date into the same numerical format that Plotly uses internally for dates.
# This is done by getting the timestamp value and converting it to milliseconds.
event_timestamp = pd.to_datetime("2023-02-01").timestamp() * 1000

fig_timeseries.add_vline(
    x=event_timestamp,  # Use the numerical timestamp
    line_dash="dash",
    line_color="red",
    annotation_text="Replika ERP Update",
    annotation_position="top left",
    annotation_font=dict(color="red"),
)

# Example for another annotation
# wysa_update_timestamp = pd.to_datetime('2022-11-15').timestamp() * 1000
# fig_timeseries.add_vline(x=wysa_update_timestamp, line_dash="dash", line_color="blue", annotation_text="Wysa v2.0 Launch")

fig_timeseries.update_layout(title_x=0.5, legend_title_text="Complaint Theme")
fig_timeseries.show()

In [33]:
# --- Visualization 1: ENHANCED Complaint Themes Over Time ---

fig_timeseries = px.line(
    monthly_trends,
    x="month",
    y="review_count",
    color="theme",
    markers=True,  # Add markers to each data point for clarity
    title="<b>Evolution of User Complaints Over Time</b>",
    labels={"month": "Month", "review_count": "Number of Negative Reviews"},
    template="plotly_white",
)

# --- Add Richer Annotations ---
event_timestamp = pd.to_datetime("2023-02-01").timestamp() * 1000

fig_timeseries.add_vline(
    x=event_timestamp, line_dash="dash", line_color="red", opacity=0.8
)

# Add a more descriptive annotation with a border and background
fig_timeseries.add_annotation(
    x=event_timestamp,
    y=monthly_trends["review_count"].max() * 0.9,  # Position annotation near the top
    text="<b>Replika ERP Update</b><br>Feb 2023",
    showarrow=True,
    arrowhead=1,
    bordercolor="red",
    borderwidth=1,
    bgcolor="white",
    opacity=0.8,
)


# --- Improve Layout and Hover Data ---
fig_timeseries.update_traces(
    hovertemplate="<b>%{y}</b> reviews for <b>%{fullData.name}</b> in %{x|%B %Y}"
)
fig_timeseries.update_layout(
    title_x=0.5, legend_title_text="Complaint Theme", font=dict(size=12)
)
fig_timeseries.show()

In [34]:
# --- Visualization 2: Thematic Dominance Over Time (Stacked Area Chart) ---

# 1. Calculate the total number of reviews for each month
monthly_totals = monthly_trends.groupby("month")["review_count"].sum().reset_index()
monthly_totals.rename(columns={"review_count": "total_monthly_reviews"}, inplace=True)

# 2. Merge the totals back into our main trends data
trends_with_totals = pd.merge(monthly_trends, monthly_totals, on="month")

# 3. Calculate the percentage share for each theme in each month
trends_with_totals["percentage_share"] = (
    trends_with_totals["review_count"] / trends_with_totals["total_monthly_reviews"]
)

# 4. Create the 100% Stacked Area Chart
fig_area = px.area(
    trends_with_totals,
    x="month",
    y="percentage_share",
    color="theme",
    groupnorm="percent",  # This is the magic that makes it a 100% stacked chart
    title="<b>The Shifting Landscape of User Frustration: Thematic Dominance Over Time</b>",
    labels={"month": "Month", "percentage_share": "Percentage of Complaints"},
    template="plotly_white",
)

# --- Add the Same Rich Annotation ---
fig_area.add_vline(
    x=pd.to_datetime("2023-02-01").timestamp() * 1000,
    line_dash="dash",
    line_color="black",  # Black stands out better on area charts
    opacity=1,
    line_width=2,
)
fig_area.add_annotation(
    x=pd.to_datetime("2023-02-01").timestamp() * 1000,
    y=90,  # Position at 90% height
    text="<b>Replika ERP<br>Update</b>",
    showarrow=True,
    arrowhead=1,
    ax=0,
    ay=-40,  # Adjust arrow
    bordercolor="black",
    borderwidth=1,
    bgcolor="white",
    opacity=0.8,
)

fig_area.update_layout(
    title_x=0.5,
    legend_title_text="Complaint Theme",
    yaxis_tickformat=".0%",  # Format y-axis as percentage
)
fig_area.show()

## Conclusion

Our analysis reveals a complex landscape of user dissatisfaction, driven by a triad of issues: **Monetization, Technical Performance, and AI Conversational Quality**. The time series analysis provides strong evidence that product decisions, such as the February 2023 Replika update, can cause immediate, massive, and multi-faceted backlash from the user base, fundamentally shifting the nature of their complaints.

In [35]:
# In the Conversational Apps notebook

# --- Final Step: Save the Enriched DataFrame ---

# Define the output filename
output_filename = "conversational_apps_themed_and_scored.csv"

# Save the DataFrame to a new CSV file in the Kaggle output directory
# This directory (/kaggle/working/) is where your new files are saved.
df.to_csv(output_filename, index=False)

print(f"Successfully saved the final, enriched dataset to: {output_filename}")
print("You can now find this file in the 'Output' section of this notebook on Kaggle.")

Successfully saved the final, enriched dataset to: conversational_apps_themed_and_scored.csv
You can now find this file in the 'Output' section of this notebook on Kaggle.
